In [126]:
from bs4 import BeautifulSoup
import requests
import time
import json

In [127]:
def extract_review_details(soup):

  reviewer_name = soup.find('a', {'class': ['a-profile', 'a-size-small']}, href=True)
  if reviewer_name:
      reviewer_name = reviewer_name.find('span', class_='a-profile-name').text.strip()
  else:
      reviewer_name = "Not Found"

  review_title_rating = soup.find('a', {'class': ['a-size-base', 'a-link-normal', 'review-title', 'a-color-base', 'review-title-content', 'a-text-bold']}, href=True)

  # print(review_title_rating)
  # print("\n\n\n\n")

  if review_title_rating:
      review_rating = review_title_rating.find('span', class_="a-icon-alt").text.strip()
  else:
      review_rating = "Not Found"

  if review_title_rating and review_title_rating.children:
      review_title = list(review_title_rating.children)[-2].text.strip() # Extremely Hard Coded and Fragile.
  else:
      review_title = "Not Found"
    
  review_date = soup.find('span', {'data-hook': 'review-date'})

  # print(review_date)
  # print("\n\n\n\n")

  if review_date:
      review_date = review_date.text.strip()
  else:
      review_date = "Not Found"

  review_data = soup.find('span', {'data-hook': 'review-body', 'class': ['a-size-base', 'review-text', 'review-text-content']})
    
  # print(review_data)
  # print("\n\n\n\n")

  if review_data:
      review_data = review_data.text.strip()
  else:
      review_data = "Not Found"

  # print(reviewer_name,review_date,review_data,review_title,review_rating)

  return {
      "Reviewer Name": reviewer_name,
      "Review Title": review_title,
      "Review Date": review_date,
      "Review Rating": review_rating,
      "Review Data": review_data
  }


In [125]:
base_url = "https://www.amazon.com/Apple-Silicone-Case-MagSafe-iPhone/product-reviews/B08L5M9L4L"
page_number = 1
all_reviews = []

In [ ]:
url = f"{base_url}?ref=cm_cr_arp_d_paging_btm_{page_number}&ie=UTF8&reviewerType=all_reviews"

while True:
  # Wait...
  print("Waiting...")
  # time.sleep(10)

  print("Executing...")

  response = requests.get(url,headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5"})
  soup = BeautifulSoup(response.content, "html.parser")


  if(response.status_code != 200):
    print("Error",response.status_code)
    print(soup)

    with open("page.html","w") as f:
        f.write(response.text)
    continue

  # Find all review sections
  reviews = soup.find_all("div", class_="a-section review aok-relative", attrs={"data-hook": "review"})

  # Extract data from reviews on this page
  for review in reviews:
    # Extract review data using functions from previous code (modify as needed)
    review_data = extract_review_details(review)
    all_reviews.append(review_data)
    
  page_number += 1

  pagination_bar = soup.find("div", id="cm_cr-pagination_bar", attrs={"data-hook":"pagination-bar"})
  print(pagination_bar)

  next_page_link = pagination_bar.find("a", class_="a-last")

  if not next_page_link or len(all_reviews)>=20:
    break

  url = next_page_link.get("href")

print("Done.")

In [111]:
# Save extracted Data:
with open("review_data.json","r+") as f:
    try:
        data = json.load(f)
    except Exception:
        data = []

    # Empty the file
    f.seek(0)
    f.write("")
    
    f.write(json.dumps(data + all_reviews))